In [ ]:
import sympy as sp
import math
import pandas as pd
from tabulate import tabulate
import numpy as np
from random import randint
import random
import matplotlib.pyplot as plt
from google.colab import data_table
from numpy.linalg import norm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Define a function to find L
def find_L(f, gradient, X, D, symbols_list):
    L = sp.symbols('L')
    phi = f.subs(list(zip(symbols_list, [x + L * d for x, d in zip(X, D)])))
    diff_phi = sp.diff(phi, L)
    L_value = sp.solve(diff_phi, L)[0]
    return L_value.evalf()

# Define the steepest_descent function
def steepest_descent(f, gradient, initial_point, symbols_list, num_classes, e=0.000001, max_iterations=7):
    X = initial_point
    df_columns = ["Iteration", "X", "f(x)", "g(x)", "norm(gx)", "d", "L"]
    df = pd.DataFrame(columns=df_columns)
    normgx_values = []

    iter = 1

    for iteration in range(max_iterations):
        # Find g(x)
        g_ = [g_i.subs(list(zip(symbols_list, X))) for g_i in gradient]
        normgx = math.sqrt(sum(g_i**2 for g_i in g_))
        normgx_values.append(normgx)

        # If normgx < e, stop
        if normgx < e:
            break

        # Find D
        D = [-g_i for g_i in g_]

        # Find L
        L = find_L(f, gradient, X, D, symbols_list)

        # Update X
        X = [x + L * d for x, d in zip(X, D)]

        # Add data into dataframe
        df = df.append({
            "Iteration": iteration + 1,
            "X": [round(p, 4) for p in X],
            "f(x)": round(f.subs(list(zip(symbols_list, X))), 4),
            "g(x)": [round(g_i, 4) for g_i in g_],
            "norm(gx)": round(normgx, 4),
            "d": [round(d_i, 4) for d_i in D],
            "L": round(L, 4)
        }, ignore_index=True)

        print("Iteration :",iter)
        iter = iter + 1
        print("normg(x) :",normgx_values[-1])
    return df, normgx_values

# Define a function to compare matrices
def compare_matrices(matrix1, matrix2):
    score = 0
    if len(matrix1) != len(matrix2) or len(matrix1[0]) != len(matrix2[0]):
        return "เมทริกซ์มีขนาดไม่เท่ากัน"

    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            if matrix1[i][j] == 1 and matrix2[i][j] == 1:
                score += 1
    return score

### Spilt Data

In [ ]:
# Read data
Y = pd.read_excel('/content/M.xlsx')
Y

In [ ]:
# Spilt X Y
X = Y.drop('Target',axis = 1 )
Y = Y['Target']

# หาค่าที่ไม่ซ้ำกันในคอลัมน์ Target_Test เพื่อใช้เป็น index ของ vectors
unique_targets = Y.unique()
num_classes = len(unique_targets)
col_Train = X.shape[1]

# Define % Test
Spilt = 0.2
TTSpilt=int(Y.shape[0]-Y.shape[0]*Spilt)

col_Train = X.shape[1]

Row_Y_train = TTSpilt
Row_Y_Test = Y.shape[0]-TTSpilt
Row_X_train = TTSpilt
Row_X_Test = Y.shape[0]-TTSpilt

X_train = X[: TTSpilt]
Y_train = Y[: TTSpilt]
X_test = X[TTSpilt :]
Y_test = Y[TTSpilt :]

In [ ]:
Target_Test = Y_test

# สร้าง vectors โดยใช้การสร้างเวกเตอร์ขนาดเท่ากับจำนวน unique_targets
vectors = {}
for i, target in enumerate(unique_targets):
    vector = [0] * len(unique_targets)
    vector[i] = 1
    vectors[target] = vector

# Create vector_list from Target_Test
vector_list = [vectors[num] for num in Target_Test]

# Create DataFrame for Target_Test
Target_Test = vector_list

In [ ]:
row = Y.shape[0]
# Find Target_Train
Target_Train = Y_train

# สร้าง vectors โดยใช้การสร้างเวกเตอร์ขนาดเท่ากับจำนวน unique_targets
vectors = {}
for i, target in enumerate(unique_targets):
    vector = [0] * len(unique_targets)
    vector[i] = 1
    vectors[target] = vector

# Create vector_list from Target_Train
vector_list = [vectors[num] for num in Target_Train]

# Create DataFrame for Target_Train
Target_Train = vector_list

X_train = X_train.values
Y_train = Y_train.values
X_test = X_test.values
Y_test = Y_test.values

### Define Node

In [ ]:
# Generate matrices W and B with random values in the range 0-10
Node = int(input("Nodes: "))
W = np.random.randint(0, 10, size=(Node, col_Train))
B = np.random.randint(0, 10, size=Node)

### Train

In [ ]:
# Create symbols_list
symbols_list = [sp.symbols(f'x{i+1}') for i in range(Node * num_classes)]

# Convert symbols_list to sympy symbols
symbols_list = [sym if isinstance(sym, sp.Symbol) else sp.Symbol(sym) for sym in symbols_list]

# Create X is array
X = np.array(symbols_list)

Z = W @ X_train.transpose() + B.reshape(-1,1)

A = 1 / (1 + math.e*(-Z))

result = A.transpose() @ X.reshape(Node, num_classes) - Target_Train
f = (np.sum(result ** 2))

gradient = [sp.diff(f, var) for var in symbols_list]

# Generate initial_point
initial_point = [random.randint(-100, 100) for _ in range( Node * num_classes )]

# Use steepest_descent and find_L
result_df, normgx_values = steepest_descent(f, gradient, initial_point, symbols_list, num_classes)

table = tabulate(result_df, headers='keys', tablefmt='fancy_grid', showindex=False)

### Plot

In [ ]:
print(table)
print("initial_point = ",initial_point)

plt.plot(normgx_values, marker='.')
plt.xlabel('Iteration')
plt.ylabel('norm(gx)')
plt.title('Norm of Gradient vs Iteration')
plt.show()

### Score

#### Train

In [ ]:
# นำ X ที่หาได้คำนวนเพื่อหาผลลัพ
min_norm_index = result_df['norm(gx)'].idxmin()
newX = np.array(result_df['X'].iloc[min_norm_index])

newX = newX.reshape(Node , num_classes)
print(newX)
O_train = A.transpose() @ newX
OO_train = math.e**O_train
OO_train

OO_sum_train = np.sum(OO_train, axis=1)
O_01_train = OO_train / OO_sum_train[:, np.newaxis]

In [ ]:
result_train = []

for val in O_01_train:
    max_val_index = np.argmax(val)
    result_train.append([1 if i == max_val_index else 0 for i in range(len(val))])

df_result_train = pd.DataFrame(result_train)

In [ ]:
print("คะแนนที่ได้:", compare_matrices(result_train, Target_Train),"/", Row_Y_train)
Accuracy = (compare_matrices(result_train, Target_Train) / Row_Y_train)
print(f"Accuracy Score = {Accuracy}")

#### Test

In [ ]:
Z_test = W @ X_test.transpose() + B.reshape(-1,1)

A_test = 1 / (1 + math.e*(-Z_test))

O_test = A_test.transpose() @ newX
OO_test = math.e**O_test
OO_test

OO_sum_test = np.sum(OO_test, axis=1)
O_01_test = OO_test / OO_sum_test[:, np.newaxis]

In [ ]:
result_test = []

for val in O_01_test:
    max_val_index = np.argmax(val)
    result_test.append([1 if i == max_val_index else 0 for i in range(len(val))])

df_result_test = pd.DataFrame(result_test)

In [ ]:
print("คะแนนที่ได้:", compare_matrices(result_test, Target_Test),"/", Row_Y_Test)
Accuracy = (compare_matrices(result_test, Target_Test) / Row_Y_Test)
print(f"Accuracy Score = {Accuracy}")